In [128]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
from src.modules import conf, fit, spec

In [129]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
data = fit.load_pcls(
        conf["Athlete"]["name"],
        conf["Athlete"]["activity_type"],
        conf["Paths"]["pcl"],
    )

100%|██████████| 363/363 [00:00<00:00, 724.19it/s]


In [131]:
from src.heuristics import random_shooting
from src.heuristics import genetic_optimization

In [132]:
r_cross = conf['GO']['r_cross']
r_mut = conf['GO']['r_mut']

In [155]:
train_df, test_df = fit.get_train_test_df(data=data)

In [159]:
len(train_df),len(test_df),len(train_df.columns),len(test_df.columns)

(542867, 128949, 77, 77)

In [160]:
full_columns = train_df.columns

In [161]:
_,_,rmse,cols = random_shooting.random_shoot(train_df=train_df,
                          test_df=test_df,
                          hmax=5,
                          endog='enhanced_speed',
                          )

100%|██████████| 5/5 [00:17<00:00,  3.41s/it]


In [ ]:
genetic_optimization.genetic_algorithm(1, r_cross,r_mut ,cols,train_df,test_df,full_columns,'enhanced_speed')

In [ ]:
genetic_optimization.mutation(cols[0],r_mut,full_columns)


In [ ]:
def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif

In [ ]:
cols[0]